<a href="https://colab.research.google.com/github/Igprad01/Sentimen-Analisis/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANALISIS SENTIMEN CASE STUDY OPINI PUBLIK TWITTER

INPUT DATA SET

In [84]:
import pandas as pd

# load data
files = '/content/sample_data/sexual_harassment gabung.csv'
df = pd.read_csv(files)
df.head()
# display(df)


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.870000e+18,Thu Dec 26 20:36:42 +0000 2024,0,@Freya_JKT48 oke aku minta maaf ya Fre sudah r...,1.870000e+18,NaN,Freya_JKT48,in,"Depok, Jawa Barat",0,0,0,https://x.com/WildXHand/status/187238104506640...,1.760000e+18,WildXHand
1,1.860000e+18,Mon Nov 25 05:27:16 +0000 2024,5,@DomLica_ @M_OlineJKT48 @AU_LiaJKT48 @CErine_J...,1.860000e+18,https://pbs.twimg.com/tweet_video_thumb/GdNPMV...,DomLica_,in,NaN,0,1,0,https://x.com/bound2vall/status/18609181535362...,1.650000e+18,bound2vall
2,1.860000e+18,Mon Nov 11 14:40:41 +0000 2024,0,@polisiidol @S_FloraJKT48 @S_GraciaJKT48 @offi...,1.860000e+18,NaN,dek_abelll,in,NaN,0,1,0,https://x.com/dek_abelll/status/18559839966788...,1.590000e+18,dek_abelll
3,1.850000e+18,Sun Nov 03 21:00:40 +0000 2024,2,@polisiidol2nd @reemblongg @officialJKT48 @Mel...,1.850000e+18,NaN,polisiidol2nd,in,NaN,0,0,0,https://x.com/ramadhannnn1999/status/185318051...,1.800000e+18,ramadhannnn1999
4,1.840000e+18,Wed Oct 09 11:12:09 +0000 2024,0,@Pandora_Box1412 @Rebornian48 Muka dia udh ter...,1.840000e+18,NaN,Pandora_Box1412,in,NaN,0,0,0,https://x.com/BagussPermana/status/18439727178...,1.550000e+18,BagussPermana


In [85]:
#tampil data
df_text = pd.DataFrame({
    'Original_Text': df['full_text'],
})

display(df_text)

,Original_Text
0,@Freya_JKT48 oke aku minta maaf ya Fre sudah r...
1,@DomLica_ @M_OlineJKT48 @AU_LiaJKT48 @CErine_J...
2,@polisiidol @S_FloraJKT48 @S_GraciaJKT48 @offi...
3,@polisiidol2nd @reemblongg @officialJKT48 @Mel...
4,@Pandora_Box1412 @Rebornian48 Muka dia udh ter...
...,...
374,@polisiidol Bacot wibuu https://t.co/XN8fDQmcpA
375,@polisiidol Soalnya emg lu yang fitnah dodol
376,@polisiidol setuju si
377,@polisiidol https://t.co/9Q8KtyO4uO


DETEKSI DUPLIKASI DATA

In [86]:
#duplicate data
df_duplicates = df[df.duplicated('full_text')]
#delete duplicates
df = df.drop_duplicates(subset='full_text')
display(df)
#download

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.870000e+18,Thu Dec 26 20:36:42 +0000 2024,0,@Freya_JKT48 oke aku minta maaf ya Fre sudah r...,1.870000e+18,NaN,Freya_JKT48,in,"Depok, Jawa Barat",0,0,0,https://x.com/WildXHand/status/187238104506640...,1.760000e+18,WildXHand
1,1.860000e+18,Mon Nov 25 05:27:16 +0000 2024,5,@DomLica_ @M_OlineJKT48 @AU_LiaJKT48 @CErine_J...,1.860000e+18,https://pbs.twimg.com/tweet_video_thumb/GdNPMV...,DomLica_,in,NaN,0,1,0,https://x.com/bound2vall/status/18609181535362...,1.650000e+18,bound2vall
2,1.860000e+18,Mon Nov 11 14:40:41 +0000 2024,0,@polisiidol @S_FloraJKT48 @S_GraciaJKT48 @offi...,1.860000e+18,NaN,dek_abelll,in,NaN,0,1,0,https://x.com/dek_abelll/status/18559839966788...,1.590000e+18,dek_abelll
3,1.850000e+18,Sun Nov 03 21:00:40 +0000 2024,2,@polisiidol2nd @reemblongg @officialJKT48 @Mel...,1.850000e+18,NaN,polisiidol2nd,in,NaN,0,0,0,https://x.com/ramadhannnn1999/status/185318051...,1.800000e+18,ramadhannnn1999
4,1.840000e+18,Wed Oct 09 11:12:09 +0000 2024,0,@Pandora_Box1412 @Rebornian48 Muka dia udh ter...,1.840000e+18,NaN,Pandora_Box1412,in,NaN,0,0,0,https://x.com/BagussPermana/status/18439727178...,1.550000e+18,BagussPermana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1.750000e+18,Thu Feb 08 04:45:21 +0000 2024,0,@polisiidol Bacot wibuu https://t.co/XN8fDQmcpA,1.760000e+18,https://pbs.twimg.com/media/GFye9HWbwAATvv0.jpg,polisiidol,in,konoha,0,0,0,https://x.com/Aearshmallow_/status/17554527341...,1.650000e+18,Aearshmallow_
375,1.750000e+18,Thu Feb 08 06:12:53 +0000 2024,0,@polisiidol Soalnya emg lu yang fitnah dodol,1.760000e+18,NaN,polisiidol,in,why u wanna know sm,0,0,0,https://x.com/aldenlatif30/status/175547476372...,1.750000e+18,aldenlatif30
376,1.750000e+18,Tue Feb 06 12:43:22 +0000 2024,0,@polisiidol setuju si,1.750000e+18,NaN,polisiidol,in,"Brebes, Indonesia",0,0,0,https://x.com/UtomoAlfat91170/status/175484825...,1.680000e+18,UtomoAlfat91170
377,1.750000e+18,Wed Feb 07 01:15:37 +0000 2024,0,@polisiidol https://t.co/9Q8KtyO4uO,1.760000e+18,https://pbs.twimg.com/media/GFslXeHaYAAdao3.jpg,polisiidol,qme,NaN,0,0,0,https://x.com/hotib48/status/1755037567358263564,1.670000e+18,hotib48


PRE PROSESSING DATA

In [87]:
#case folding
df_text = pd.DataFrame({
    'Original_Text': df['full_text'],
    'Lowercased_Text': df['full_text'].str.lower(),
    # 'tweet_url': df['tweet_url']
})

display(df_text)

,Original_Text,Lowercased_Text
0,@Freya_JKT48 oke aku minta maaf ya Fre sudah r...,@freya_jkt48 oke aku minta maaf ya fre sudah r...
1,@DomLica_ @M_OlineJKT48 @AU_LiaJKT48 @CErine_J...,@domlica_ @m_olinejkt48 @au_liajkt48 @cerine_j...
2,@polisiidol @S_FloraJKT48 @S_GraciaJKT48 @offi...,@polisiidol @s_florajkt48 @s_graciajkt48 @offi...
3,@polisiidol2nd @reemblongg @officialJKT48 @Mel...,@polisiidol2nd @reemblongg @officialjkt48 @mel...
4,@Pandora_Box1412 @Rebornian48 Muka dia udh ter...,@pandora_box1412 @rebornian48 muka dia udh ter...
...,...,...
374,@polisiidol Bacot wibuu https://t.co/XN8fDQmcpA,@polisiidol bacot wibuu https://t.co/xn8fdqmcpa
375,@polisiidol Soalnya emg lu yang fitnah dodol,@polisiidol soalnya emg lu yang fitnah dodol
376,@polisiidol setuju si,@polisiidol setuju si
377,@polisiidol https://t.co/9Q8KtyO4uO,@polisiidol https://t.co/9q8ktyo4uo


In [88]:
#cleaning text (Sementara)
import re
def clean_text(text):
    # Menghapus URL
    text = re.sub(r'#', '', text)  # Remove hashtag symbol
    text = re.sub(r'http\S+|www.\S+', '', text ) #remove URl maupun Tautan
    text = re.sub(r'[^\w\s]', '', text) # Rewmove Tanda baca dan karakter khusus
    text = re.sub(r'\d','', text) #remove Angka
    text = re.sub(r'@|w\s','',text) #remove Karakter @ dan pola khusus
    text = re.sub(r'\s+', ' ', text) #remove spasi ganda
    text = text.strip() #remove spasi awal dan akhir
    return text


df_text['Cleaned_Text'] = df_text['Lowercased_Text'].apply(clean_text)
display(df_text[['Lowercased_Text','Cleaned_Text']])


,Lowercased_Text,Cleaned_Text
0,@freya_jkt48 oke aku minta maaf ya fre sudah r...,freya_jkt oke aku minta maaf ya fre sudah riwe...
1,@domlica_ @m_olinejkt48 @au_liajkt48 @cerine_j...,domlica_ m_olinejkt au_liajkt cerine_jkt semen...
2,@polisiidol @s_florajkt48 @s_graciajkt48 @offi...,polisiidol s_florajkt s_graciajkt officialjkt ...
3,@polisiidol2nd @reemblongg @officialjkt48 @mel...,polisiidolnd reemblongg officialjkt melodylaks...
4,@pandora_box1412 @rebornian48 muka dia udh ter...,pandora_box rebornian muka dia udh terpampang ...
...,...,...
374,@polisiidol bacot wibuu https://t.co/xn8fdqmcpa,polisiidol bacot wibuu
375,@polisiidol soalnya emg lu yang fitnah dodol,polisiidol soalnya emg lu yang fitnah dodol
376,@polisiidol setuju si,polisiidol setuju si
377,@polisiidol https://t.co/9q8ktyo4uo,polisiidol


In [89]:
!pip install sastrawi
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
#Tokenization
df_text['Tokenized_Text'] = df_text['Cleaned_Text'].apply(word_tokenize)
#display(df_text)
display(df_text[['Cleaned_Text','Tokenized_Text']])
df_text.to_csv('df_text_token_sexual_harasment.csv', index=False)

,Cleaned_Text,Tokenized_Text
0,freya_jkt oke aku minta maaf ya fre sudah riwe...,"[freya_jkt, oke, aku, minta, maaf, ya, fre, su..."
1,domlica_ m_olinejkt au_liajkt cerine_jkt semen...,"[domlica_, m_olinejkt, au_liajkt, cerine_jkt, ..."
2,polisiidol s_florajkt s_graciajkt officialjkt ...,"[polisiidol, s_florajkt, s_graciajkt, official..."
3,polisiidolnd reemblongg officialjkt melodylaks...,"[polisiidolnd, reemblongg, officialjkt, melody..."
4,pandora_box rebornian muka dia udh terpampang ...,"[pandora_box, rebornian, muka, dia, udh, terpa..."
...,...,...
374,polisiidol bacot wibuu,"[polisiidol, bacot, wibuu]"
375,polisiidol soalnya emg lu yang fitnah dodol,"[polisiidol, soalnya, emg, lu, yang, fitnah, d..."
376,polisiidol setuju si,"[polisiidol, setuju, si]"
377,polisiidol,[polisiidol]


NORMALISASI


In [51]:
#Stopwords
stop_words = set(stopwords.words('indonesian'))
df_text['Stopwords_removal'] = df_text['Tokenized_Text'].apply(lambda x: [word for word in x if word not in stop_words])
df_text.head()
#download
df_text.to_csv('df_text_stopwords_removal.csv', index=False)

In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df_text['Stemming'] = df_text['Stopwords_removal'].apply(lambda x: [stemmer.stem(word) for word in x])
df_text.head()

,Original_Text,Lowercased_Text,Cleaned_Text,Tokenized_Text,Filtered_Text,Stopwords_removal,Stemming
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,om jot liat deh si up girls ngajak @rfritzy_jk...,om jot liat deh si up girls ngajak rfritzy_jkt...,"[om, jot, liat, deh, si, up, girls, ngajak, rf...","[om, jot, liat, deh, si, up, girls, ngajak, rf...","[om, jot, liat, deh, si, up, girls, ngajak, rf...","[om, jot, liat, deh, si, up, girls, ngajak, rf..."
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...,rfritzy_jkt kalo oshinya jot masih ga apa apa ...,"[rfritzy_jkt, kalo, oshinya, jot, masih, ga, a...","[rfritzy_jkt, kalo, oshinya, jot, ga, nder, hehe]","[rfritzy_jkt, kalo, oshinya, jot, ga, nder, hehe]","[rfritzy jkt, kalo, oshinya, jot, ga, nder, hehe]"
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt yaitu membuat para jot sadar...,"[harapan, untuk, jkt, yaitu, membuat, para, jo...","[harapan, jkt, jot, sadar, fans, apaapa, gaaka...","[harapan, jkt, jot, sadar, fans, apaapa, gaaka...","[harap, jkt, jot, sadar, fans, apaapa, gaakan,..."
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...,_rybh maffi_f auwia_jkt itu urusan internal jo...,"[_rybh, maffi_f, auwia_jkt, itu, urusan, inter...","[_rybh, maffi_f, auwia_jkt, urusan, internal, ...","[_rybh, maffi_f, auwia_jkt, urusan, internal, ...","[rybh, maffi f, auwia jkt, urus, internal, jot..."
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...,gagitugie___ maffi_f auwia_jkt menunjukan kalo...,"[gagitugie___, maffi_f, auwia_jkt, menunjukan,...","[gagitugie___, maffi_f, auwia_jkt, menunjukan,...","[gagitugie___, maffi_f, auwia_jkt, menunjukan,...","[gagitugie, maffi f, auwia jkt, tunjuk, kalo, ..."


In [ ]:
#dopwnload
df_text.to_csv('df_text_cleaning.csv', index=False)

PELABELAN DATA

In [ ]:
# Pelabelan Data


In [ ]:
# extrasi fitur

In [ ]:
# klasifikasi sentimen

In [ ]:
# evaluasi hasil


In [ ]:
# visualisasi dan hasilnya